# **Custom Model (Small version of VGG) for Detecting MRI anamolies in Brain using Brain MRI dataset from Kaggle**

In [ ]:
import torch
from torch import nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
print(torch.__version__)
print(torchvision.__version__)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
import requests
import kagglehub
import zipfile
from pathlib import Path
import os
import shutil
from sklearn.model_selection import train_test_split

# Download dataset using kagglehub
path = kagglehub.dataset_download("rm1000/brain-tumor-mri-scans")
print(f"Path to dataset: {path}")

# Define paths for training and test sets
train_path = "/content/train"
test_path = "/content/test"

# Check if train and test sets already exist
if os.path.exists(train_path) and os.path.exists(test_path):
    print("Train and test sets already exist.")
else:
    # Create directories for training and testing sets
    os.makedirs(train_path, exist_ok=True)
    os.makedirs(test_path, exist_ok=True)

    # Split each classification into training and test sets
    for class_name in os.listdir(path):
        class_dir = os.path.join(path, class_name)

        # Skip if not a directory
        if not os.path.isdir(class_dir):
            continue

        # List all images in the class directory
        images = os.listdir(class_dir)

        # Split into training and testing sets
        train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

        # Create subdirectories for the class in train and test folders
        os.makedirs(os.path.join(train_path, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_path, class_name), exist_ok=True)

        # Move training images
        for image in train_images:
            shutil.move(os.path.join(class_dir, image), os.path.join(train_path, class_name, image))

        # Move testing images
        for image in test_images:
            shutil.move(os.path.join(class_dir, image), os.path.join(test_path, class_name, image))

    print("Dataset successfully divided into training and test sets!")

In [ ]:
train_transform_trivial= transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.TrivialAugmentWide(num_magnitude_bins=31),
    transforms.ToTensor()
])
test_transform_simple= transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor()
])
train_data_augmented=datasets.ImageFolder(root=train_path, transform=train_transform_trivial)
test_data_simple=datasets.ImageFolder(root=test_path, transform=test_transform_simple)
from torch.utils.data import DataLoader
BATCH_SIZE=50
train_dataloader_augmented=DataLoader(dataset=train_data_augmented, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader_simple=DataLoader(dataset=test_data_simple, batch_size=BATCH_SIZE, shuffle=False)
img,label=next(iter(train_dataloader_augmented))

In [ ]:
class TinyVGG(nn.Module):
  def __init__(self,input_shape:int,hidden_units:int,output_shape:int)->None:
    super().__init__()
    self.conv_block_1=nn.Sequential(
        nn.Conv2d(in_channels=input_shape, out_channels=hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.conv_block_2=nn.Sequential(
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units, out_channels=hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2)
    )
    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*16*16, out_features=output_shape)
    )
  def forward(self,x):
    return self.classifier(self.conv_block_2(self.conv_block_1(x)))
torch.manual_seed(42)
model=TinyVGG(input_shape=3, hidden_units=10, output_shape=len(train_data_augmented)).to(device)
model

In [ ]:
def train_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module,
               optimizer:torch.optim.Optimizer):
  model.train()
  train_loss, train_acc=0,0
  for batch, (X,y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    y_pred=model(X)
    loss=loss_fn(y_pred,y)
    train_loss+=loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    y_pred_class=torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
    train_acc+=(y_pred_class==y).sum().item()/len(y_pred)
    train_loss/=len(dataloader)
  train_acc/=len(dataloader)
  return train_loss, train_acc
def test_step(model:torch.nn.Module,
               dataloader:torch.utils.data.DataLoader,
               loss_fn:torch.nn.Module):
  model.eval()
  test_loss, test_acc=0,0
  with torch.inference_mode():
    for batch, (X,y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      test_pred_logits=model(X)
      loss=loss_fn(test_pred_logits,y)
      test_loss+=loss.item()
      test_pred_labels=test_pred_logits.argmax(dim=1)
      test_acc+=(test_pred_labels==y).sum().item()/len(test_pred_labels)
  test_loss/=len(dataloader)
  test_acc/=len(dataloader)
  return test_loss, test_acc
from tqdm.auto import tqdm
def train(model:torch.nn.Module,
          train_dataloader:torch.utils.data.DataLoader,
          test_dataloader:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:torch.nn=nn.CrossEntropyLoss(),
          epochs:int=5):
  results={"train_loss":[],
          "train_acc":[],
          "test_loss":[],
          "test_acc":[]}
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc=train_step(model=model,
                                      dataloader=train_dataloader,
                                      loss_fn=loss_fn,
                                      optimizer=optimizer)
    test_loss, test_acc=test_step(model=model,
                                    dataloader=test_dataloader,
                                    loss_fn=loss_fn)
    print(f"Epoch: {epoch} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.4f}")
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  return results

In [ ]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=model.parameters(), lr=0.001)
from timeit import default_timer as timer
start_time=timer()
results=train(model=model,
              train_dataloader=train_dataloader_augmented,
              test_dataloader=test_dataloader_simple,
              optimizer=optimizer,
              loss_fn=loss_fn,
              epochs=5)
end_time=timer()
print(f"Total training time {end_time - start_time:.3f} seconds")

In [ ]:
import requests
custom_image_path="/content/s1.png"
custom_image_transform=transforms.Compose([
    transforms.Resize(size=(64,64))
])
def pred_and_plot_image(model:torch.nn.Module,
                         image_path:str,
                         class_names:list[str]=None,
                         transform=None,
                         device=device):
  target_image=torchvision.io.read_image(str(image_path)).type(torch.float32)/255.0
  if transform:
    target_image=transform(target_image)
  model.to(device)
  model.eval()
  with torch.inference_mode():
    target_image_pred=model(target_image.unsqueeze(0).to(device))
  target_image_pred_prob=torch.softmax(target_image_pred, dim=1)
  target_image_pred_label=torch.argmax(target_image_pred_prob, dim=1)
  plt.imshow(target_image.squeeze().permute(1,2,0))
  if class_names:
    title=f"Pred: {class_names[target_image_pred_label.cpu()]} | Prob: {target_image_pred_prob.max():.3f}"
  else:
    title=f"Pred: {target_image_pred_label} | Prob: {target_image_pred_prob.max():.3f}"
  plt.title(title)
  plt.axis(False)
pred_and_plot_image(model=model,
                  image_path=custom_image_path,
                  class_names=train_data_augmented.classes,
                  transform=custom_image_transform,
                  device=device)